<a href="https://colab.research.google.com/github/srikarraju/eGrocery_Demand_Prediction/blob/main/PPO_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboardX

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import random
import pandas as pd
from tensorboardX import SummaryWriter

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std, device):
        super(ActorCritic, self).__init__()
	#actor
        self.actor =  nn.Sequential(
                nn.Linear(state_dim, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, action_dim),
		nn.ReLU()
                )
        # critic
        self.critic = nn.Sequential(
                nn.Linear(state_dim, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 1)
                )
        self.device = device
        self.action_var = torch.full((action_dim,), action_std*action_std).to(self.device)

    def forward(self):
        raise NotImplementedError

    def act(self, state, memory):
        action_mean = self.actor(state)
        cov_mat = torch.diag(self.action_var).to(self.device)

        dist = MultivariateNormal(action_mean, cov_mat)
        action = dist.sample()
        action_logprob = dist.log_prob(action)

        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(action_logprob)

        return action.detach()

    def evaluate(self, state, action):
        action_mean = self.actor(state)

        action_var = self.action_var.expand_as(action_mean)
        cov_mat = torch.diag_embed(action_var).to(self.device)

        dist = MultivariateNormal(action_mean, cov_mat)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_value = self.critic(state)

        return action_logprobs, torch.squeeze(state_value), dist_entropy


In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip, device):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        self.device = device

        self.policy = ActorCritic(state_dim, action_dim, action_std,self.device).to(self.device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)

        self.policy_old = ActorCritic(state_dim, action_dim, action_std,self.device).to(self.device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()

    def select_action(self, state, memory):
        state = torch.FloatTensor(state.reshape(1, -1)).to(self.device)
        return self.policy_old.act(state, memory).cpu().data.numpy().flatten()

    def update(self, memory):
        # Monte Carlo estimate of rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(self.device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(memory.states).to(self.device), 1).detach()
        old_actions = torch.squeeze(torch.stack(memory.actions).to(self.device), 1).detach()
        old_logprobs = torch.squeeze(torch.stack(memory.logprobs), 1).to(self.device).detach()

        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            loss = loss.type(torch.cuda.FloatTensor)

            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())


In [ ]:
class EGroceryEnv():

	def __init__(self, df=pd.DataFrame({0:[0]}), products_count=10, features=['a'], shelf_life=[1], wastage_cost=[1], shortage_cost=[1]):
		super(EGroceryEnv, self).__init__()

		self.df = df
		self.products_count = products_count
		self.shelf_life = shelf_life
		self.features = features
		self.wastage_cost = wastage_cost
		self.shortage_cost = shortage_cost
		self.current_step = 0
		self.wastage_track = list([])
		self.shortage_track = list([])
		self.reward_track = list([])

		#variables to track shartage and wastage
		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))

		#Define Stock
		self.stock = list([])
		for i in range(self.products_count):
			self.stock.append([0])


	def _next_observation(self):

		obs  = self.df.loc[self.current_step,self.features]

		st_temp = list([])
		for i in range(len(self.stock)):
			for j in range(1,min(int(self.stock[i][0])+1,5)):
				st_temp.append(self.stock[i][j])
			if(self.stock[i][0]==5):
				st_temp.append(self.stock[i][4])
			elif(self.stock[i][0]<5):
				for j in range(int(self.stock[i][0])+1,6):
					st_temp.append(0)
			else:
				st_temp.append(np.sum(self.stock[i][5:int(self.stock[i][0])+1]))
		obs = list(obs) + list(st_temp) + list(self.shelf_life)
		return obs

	def _take_action(self, action):
		#Add products to the current stocks
		for i in range(self.products_count):
			if(len(self.stock[i])<self.shelf_life[i]):
				for j in range(len(self.stock[i]),int(self.shelf_life[i])):
					self.stock[i].append(0)
			self.stock[i].append(action[i])
			self.stock[i][0]=self.shelf_life[i]



		#Fullfill demand
		prods = ['prod'+str(i) for i in [8,11,15,17,94,95,96,110,112,128]]
		demand = self.df.loc[self.current_step+1,prods]
		for i in range(self.products_count):
			for j in range(1,int(self.stock[i][0])+1):
				if(self.stock[i][j]>=demand[i]):
					self.stock[i][j] = self.stock[i][j] - demand[i]
					demand[i] = 0
					break
				else:
					demand[i] = demand[i] - self.stock[i][j]
					self.stock[i][j] = 0
			if(demand[i]>0):
				self.shortage[i]=demand[i]

		#Update shelf life and find out wastage
		for i in range(self.products_count):
			self.stock[i][0] = self.stock[i][0] -1
			if(self.stock[i][1]>0):
				self.wastage[i] = self.stock[i][1]
			for j in range(1,int(self.stock[i][0])+1):
				self.stock[i][j] = self.stock[i][j+1]
			self.stock[i].pop()


	def step(self, action):
	        # update stock, fullfill demand and calculate shortage and wastage
		quantity = [6, 10, 15, 4, 6, 2, 7, 50, 2, 30]
		action1 = [0]*self.products_count
		for i in range(len(action)):
			action1[i] = action[i]*quantity[i]
		self._take_action(action1)
		self.action = action

		#increment step
		self.current_step += 1


		reward = -1*(np.matmul(self.wastage_cost,self.wastage.transpose())+np.matmul(self.shortage_cost,self.shortage.transpose()))
		self.reward = reward
		done = (self.current_step < 0) or (self.current_step > self.df.shape[0]-2)

		obs = self._next_observation()

		self.wastage_track.append(np.sum(self.wastage))
		self.shortage_track.append(np.sum(self.shortage))
		self.reward_track.append(np.abs(self.reward))


		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))

		return obs, reward, done, {}

	def reset(self):
		# Reset the state of the environment to an initial state
		self.current_step = 0
		self.shortage = np.array(list([0]*self.products_count))
		self.wastage = np.array(list([0]*self.products_count))
		self.stock = list([])
		for i in range(self.products_count):
			self.stock.append([0])

		return [0]*len(self.features) + [0]*6*self.products_count

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_trainx.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/data/final_data_testx.csv')

products_count = 10

avg_f7 = ['prod'+str(i)+'avg7' for i in [8,11,15,17,94,95,96,110,112,128]]
avg_f15 = ['prod'+str(i)+'avg15' for i in [8,11,15,17,94,95,96,110,112,128]]
avg_f30 = ['prod'+str(i)+'avg30' for i in [8,11,15,17,94,95,96,110,112,128]]

features = ['month', 'monthday', 'weekday'] + avg_f7 + avg_f15 + avg_f30

print(features)

shelf_life = np.array([4, 3, 5, 10, 7, 2, 1, 3, 8, 6], dtype=np.float32)

wastage_cost = np.array([1]*products_count, dtype=np.float16)
shortage_cost = np.array([1]*products_count, dtype=np.float16)

action_std = 0.1
eps_clip = 0.2
gamma = 0.99

lr = 0.00001
betas = (0.9, 0.999)
K_epochs = 5

update_timestep = 100
time_step=0
running_reward = 0

state_dim = len(features) + 6*products_count
action_dim = products_count

env = EGroceryEnv(df_train, products_count, features, shelf_life, wastage_cost, shortage_cost)

memory = Memory()
ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip, device)

writer = SummaryWriter()

Total_reward = []


['month', 'monthday', 'weekday', 'prod8avg7', 'prod11avg7', 'prod15avg7', 'prod17avg7', 'prod94avg7', 'prod95avg7', 'prod96avg7', 'prod110avg7', 'prod112avg7', 'prod128avg7', 'prod8avg15', 'prod11avg15', 'prod15avg15', 'prod17avg15', 'prod94avg15', 'prod95avg15', 'prod96avg15', 'prod110avg15', 'prod112avg15', 'prod128avg15', 'prod8avg30', 'prod11avg30', 'prod15avg30', 'prod17avg30', 'prod94avg30', 'prod95avg30', 'prod96avg30', 'prod110avg30', 'prod112avg30', 'prod128avg30']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for epoch in range(1,5):
    obs = env.reset()
    obs = np.array(obs)
    running_reward = 0

    for i in range(50):
        time_step +=1

        action = ppo.select_action(obs, memory)
        action = action
        action = list(action.astype(int))
        obs, reward, done, _ = env.step(action)
        obs = np.array(obs)

        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        if(time_step%update_timestep==0):
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0
        running_reward+=reward

        writer.add_scalar("reward", reward, time_step)

        if(done==1):
            break

    Total_reward.append(running_reward)



    if(epoch%1)==0:
        torch.save(ppo.policy.state_dict(), '/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/model_{}.pth'.format(epoch))
        pd.DataFrame({'Epoch':epoch,'Reward ': Total_reward}).to_csv('/content/drive/MyDrive/Colab Notebooks/RL_Project/Models/ppo_based/PPOBased/runs/Training.csv',index=False)

    print('Epoch ', epoch, ' Reward ', running_reward)


writer.close()

Epoch  1  Reward  -15763.0
